In [ ]:
!pip install validators
!pip install pygetpapers==1.1.6
import os
import numpy as np
import validators
import csv
import requests
import xml.etree.ElementTree as ET
import time
from bs4 import BeautifulSoup

     |████████████████████████████████| 40 kB 5.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 46 kB 2.0 MB/s 
     |████████████████████████████████| 81 kB 5.4 MB/s 
     |████████████████████████████████| 86 kB 3.3 MB/s 
  Created wheel for dict2xml: filename=dict2xml-1.7.1-py3-none-any.whl size=6930 sha256=651af2dfe1cc9a45944a096a2a291c5e80d974eb5d8d1940b6ebd47719a2b37c
  Stored in directory: /root/.cache/pip/wheels/8e/ad/d7/b1cd889d18b22fa7d175a92006236862743f39cdefda238397
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6066 sha256=fc067f7d1b157f3977f4c4c6acf7cb09c42683c45c316447e06d7508b21ff01b
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36cebfdddfe059f50e019aa
Successfully built dict2xml sgmllib3k


In [ ]:
!pygetpapers -q terpene -x -k 5 -o junk

INFO: Total Hits are 13968
1it [00:00, 1007.04it/s]
INFO: Saving XML files to /content/junk/*/fulltext.xml
100% 5/5 [00:14<00:00,  2.86s/it]


In [ ]:
%cd /content/drive/MyDrive/ctree_folder/

/content/drive/MyDrive/ctree_folder


In [ ]:
!pygetpapers  -q 'terpene' -x -k 10 -o terpene_pmcids 

INFO: Total Hits are 13968
1it [00:00, 579.24it/s]
INFO: Saving XML files to /content/terpene_pmcids/*/fulltext.xml
100% 10/10 [00:18<00:00,  1.84s/it]


In [ ]:
!pygetpapers -v

INFO: You are running pygetpapers version 1.1.4


###vaishnavi

#original

In [ ]:
#terpene_folder_path = '/content/drive/MyDrive/ctree_folder/2022_03_17_13_58_44/'
terpene_folder_path = '/content/PMCID_test/'
names= np.sort(os.listdir(terpene_folder_path))
PMCIDs = [item for item in names if item[0] == 'P']
for pmcid in PMCIDs[:3]:
  print('pmcid',pmcid)
  #creating a parent PMCXXXXXXX directory
  path = '/content/Alex_30/'

  tree1 = path + pmcid +'/'
  if not os.path.exists(tree1):
    os.makedirs(tree1)

  #creating a figure directory inside PMCXXXXXXX directory
  FIGURE = 'figure'  
  tree2 = tree1 + FIGURE +'/' 
  if not os.path.exists(tree2):
    os.makedirs(tree2)

  #parsing the xml to find fig no are respective Id 
  path_xml = terpene_folder_path +'/'+ pmcid +'/'+ 'fulltext.xml'
  tree = ET.parse(path_xml)
  #print(tree)
  # get root element
  root = tree.getroot()
 
  # collecting figure id's and  graphic id's
  figures_id = []
  graphic_id = []
  caption_text=[]
  italics = []
  # create functions -----do it 
  for fig in root.iter('fig'):
      # appending figure_id
      figures_id.append(fig.attrib.get('id'))
      # appending graphic_id
      gid = fig.find('graphic')
      graphic_id.append(gid.attrib.get('{http://www.w3.org/1999/xlink}href'))
      # extracting caption text and appending it
      caption_content = fig.find('caption')
      xmlstr = ET.tostring(caption_content, encoding='utf8', method='xml')
      soup = BeautifulSoup(xmlstr, features='lxml')
      text = soup.get_text(separator=" ")
      paragraph_text = text.replace('\n', ' ')
      caption_text.append(paragraph_text)
      italic_list =[]
      for italic in caption_content.iter('italic'):
        #print(italic.text)
        italic_list.append(italic.text)
      italics.append(italic_list)

      

  print('figures_id',figures_id) 
  print('graphic_id',graphic_id)
  print('caption_text',caption_text)
  print('italics',italics)
  print(len(figures_id),len(graphic_id),len(caption_text))
  
  # creating fig, image and caption folder within each subtree.
  for j,fig_id in enumerate(figures_id) :
        tree3 = tree2 + fig_id +'/' 
        if not os.path.exists(tree3):
          os.makedirs(tree3)
        
        download_fig_url = 'https://europepmc.org/articles/'+ pmcid + '/bin/' + graphic_id[j] +'.jpg'
        
        IMAGE = 'image'
        CAPTION = 'caption'
        tree4 = tree3 + IMAGE + '/'
        tree5 = tree3+ CAPTION + '/'
        if not os.path.exists(tree4):
          os.makedirs(tree4)
        if not os.path.exists(tree5):
          os.makedirs(tree5)  
        
        fig_jpg = requests.get(download_fig_url)
        time.sleep(5)
        NAME_FIG ='raw.jpg'
        NAME_CAPTION = 'raw_text.txt'
        NAME_CAPTION_XML = 'raw_xml'
        NAME_ITALICS = 'raw_italics.txt'
        with open(tree4 + NAME_FIG, 'wb') as file:
            file.write(fig_jpg.content)

        with open(tree5 + NAME_CAPTION, 'w') as file:
            file.write(caption_text[j])

        with open(tree5 + NAME_ITALICS, 'w') as file:
              for i in range(len(italics[j])):
                 file.write(italics[j][i]+ '\n')

  # create functions --- do it
  # extracting xml of captions for each figure
  for fig in root.iter('fig'):
      figures_id.append(fig.attrib.get('id'))
      caption_content = fig.find('caption')
      CAPTION = 'caption'
      tree6 = tree2 + fig.attrib.get('id') + '/' + CAPTION + '/'
      NAME_CAPTION_XML = 'raw_xml'
      filename = format(tree6 + NAME_CAPTION_XML + ".xml")
      with open(filename, 'wb') as f:
          #f.write("<?xml version=\"1.0\" encoding=\"UTF-8\"?>\n")
          f.write(ET.tostring(caption_content))
      


pmcid PMC8050080
figures_id ['Fig1', 'Fig2', 'Fig3', 'Fig4', 'Fig5', 'Fig6', 'Fig7', 'Fig8']
graphic_id ['41598_2021_87740_Fig1_HTML', '41598_2021_87740_Fig2_HTML', '41598_2021_87740_Fig3_HTML', '41598_2021_87740_Fig4_HTML', '41598_2021_87740_Fig5_HTML', '41598_2021_87740_Fig6_HTML', '41598_2021_87740_Fig7_HTML', '41598_2021_87740_Fig8_HTML']
caption_text ['Terpenes Induce Cannabimimetic Tetrad Behaviors in Mice. Behavioral analysis data from Figs.  S1 - S3  represented in a radar chart format. The percent change from baseline for each terpene in each behavior was determined; the peak effect for tail flick was selected (Fig.  S1 ) while the other behaviors only had a single time point. This percent change was then normalized to the % change of 5.6\xa0mg/kg WIN55,212-2 (1.0 on the chart above). The chart demonstrates that while all terpenes induce all 4 tetrad behaviors, they do so with varying efficacy; they also vary whether the effect is stronger or weaker than the positive control W

### species are the problem find something to solve it
### **lower case text**
### **lemmatization and stemming**
### Abbreviation
### spelling different --- find a lib to solve it
### **italic for species**
### dictionary for genus, plant species ----already there
### look up into the wikidata

dictionary for genus and different species --- main idea will help to search offline



In [ ]:
!pip install scispacy
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz
import scispacy
import spacy
#import en_core_sci_sm
from spacy import displacy
import pandas as pd 

     |████████████████████████████████| 44 kB 1.9 MB/s 
     |████████████████████████████████| 71 kB 5.3 MB/s 
     |████████████████████████████████| 13.5 MB 23.2 MB/s 
     |████████████████████████████████| 6.0 MB 52.9 MB/s 
     |████████████████████████████████| 188 kB 51.1 MB/s 
     |████████████████████████████████| 42 kB 1.6 MB/s 
     |████████████████████████████████| 653 kB 54.9 MB/s 
     |████████████████████████████████| 181 kB 67.3 MB/s 
     |████████████████████████████████| 10.1 MB 46.4 MB/s 
     |████████████████████████████████| 451 kB 62.7 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      S

In [ ]:
from scispacy.abbreviation import AbbreviationDetector
from scispacy.umls_linking import UmlsEntityLinker
from collections import OrderedDict
from pprint import pprint

In [ ]:
import csv,sys
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, train_test_split 
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, recall_score, f1_score
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

True

In [ ]:
#dic_terpene = ['terpene', 'limonene',]

In [ ]:
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bionlp13cg_md-0.5.0.tar.gz

In [ ]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))


analyzer = CountVectorizer().build_analyzer()

def stemmed_words(doc):
  return (PorterStemmer().stem(w) for w in analyzer(doc))

def lemmatized_words(doc):
  return (WordNetLemmatizer().lemmatize(w) for w in analyzer(doc))


def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    #text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    text = ' '.join(word for word in analyzer(text) if word not in STOPWORDS) # delete stopwors from text
    text = ' '.join(WordNetLemmatizer().lemmatize(w) for w in analyzer(text))
    text = ' '.join(word for word in analyzer(text) if len(word) >3) 
    # text_clean = []
    # for w in text.split():
    #   if len(w)>3:
    #     if 
    #     text_clean.append(w)
        

    return text

# text =  clean_text(reader_train_data[0][0])
# text

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_sm-0.5.0.tar.gz
import en_core_sci_sm

     |████████████████████████████████| 15.9 MB 1.0 MB/s 
  Created wheel for en-core-sci-sm: filename=en_core_sci_sm-0.5.0-py3-none-any.whl size=15871046 sha256=307f1cbbf06804ff979f91db54453c3f393fee0b8f3ca640c97e997bbcec51de
  Stored in directory: /root/.cache/pip/wheels/b1/b8/7d/d26a5894db99bf0cfb9add399889738de91a608e6664f82454
Successfully built en-core-sci-sm


In [ ]:
f_train_data = open('/content/terpene/PMC8050080/figure/Fig1/caption/raw_text.txt','r')
reader_train_data = list(csv.reader(f_train_data,delimiter='\t'))
text = reader_train_data[0][0]
text_clean = clean_text(text)

In [ ]:
nlp = en_core_sci_sm.load()
doc_raw= nlp(text)
doc_clean_text = nlp(text_clean)
#displacy_image = displacy.render(doc, jupyter = True, style = ‘ent’)

**Raw text:**


Terpenes Induce Cannabimimetic Tetrad Behaviors in Mice. Behavioral analysis data from Figs.  S1 - S3  represented in a radar chart format. The percent change from baseline for each terpene in each behavior was determined; the peak effect for tail flick was selected (Fig.  S1 ) while the other behaviors only had a single time point. This percent change was then normalized to the % change of 5.6 mg/kg WIN55,212-2 (1.0 on the chart above). The chart demonstrates that while all terpenes induce all 4 tetrad behaviors, they do so with varying efficacy; they also vary whether the effect is stronger or weaker than the positive control WIN55,212-2.
Terpenes ENTITY Induce Cannabimimetic ENTITY Tetrad Behaviors ENTITY in Mice ENTITY . Behavioral ENTITY analysis data

In [ ]:
print('FOR RAW TEXT:\n')
print(text)
displacy_image = displacy.render(doc_raw, jupyter=True,style='ent')
entity_and_label = pprint(set([(X.text, X.label_) for X in doc_raw.ents]))
displacy_image, entity_and_label

FOR RAW TEXT:

Terpenes Induce Cannabimimetic Tetrad Behaviors in Mice. Behavioral analysis data from Figs.  S1 - S3  represented in a radar chart format. The percent change from baseline for each terpene in each behavior was determined; the peak effect for tail flick was selected (Fig.  S1 ) while the other behaviors only had a single time point. This percent change was then normalized to the % change of 5.6 mg/kg WIN55,212-2 (1.0 on the chart above). The chart demonstrates that while all terpenes induce all 4 tetrad behaviors, they do so with varying efficacy; they also vary whether the effect is stronger or weaker than the positive control WIN55,212-2.


{('Behavioral', 'ENTITY'),
 ('Cannabimimetic', 'ENTITY'),
 ('Mice', 'ENTITY'),
 ('S3', 'ENTITY'),
 ('Terpenes', 'ENTITY'),
 ('Tetrad Behaviors', 'ENTITY'),
 ('baseline', 'ENTITY'),
 ('behavior', 'ENTITY'),
 ('behaviors', 'ENTITY'),
 ('change', 'ENTITY'),
 ('chart', 'ENTITY'),
 ('chart format', 'ENTITY'),
 ('data', 'ENTITY'),
 ('effect', 'ENTITY'),
 ('efficacy', 'ENTITY'),
 ('induce', 'ENTITY'),
 ('peak effect', 'ENTITY'),
 ('positive control WIN55,212-2', 'ENTITY'),
 ('stronger', 'ENTITY'),
 ('tail', 'ENTITY'),
 ('terpene', 'ENTITY'),
 ('terpenes', 'ENTITY'),
 ('tetrad behaviors', 'ENTITY'),
 ('weaker', 'ENTITY')}


(None, None)

In [ ]:
print('FOR CLEAN TEXT:\n')
print(text_clean)
displacy_image = displacy.render(doc_clean_text, jupyter=True,style='ent')
entity_and_label = pprint(set([(X.text, X.label_) for X in doc_clean_text.ents]))
displacy_image, entity_and_label

FOR CLEAN TEXT:

terpene induce cannabimimetic tetrad behavior mouse behavioral analysis data represented radar chart format percent change baseline terpene behavior determined peak effect tail flick selected behavior single time point percent change normalized change 56mg win55 2122 chart chart demonstrates terpene induce tetrad behavior varying efficacy also vary whether effect stronger weaker positive control win55 2122


{('baseline', 'ENTITY'),
 ('behavioral', 'ENTITY'),
 ('cannabimimetic', 'ENTITY'),
 ('change', 'ENTITY'),
 ('chart chart', 'ENTITY'),
 ('chart format', 'ENTITY'),
 ('data', 'ENTITY'),
 ('effect', 'ENTITY'),
 ('efficacy', 'ENTITY'),
 ('flick', 'ENTITY'),
 ('induce', 'ENTITY'),
 ('mouse', 'ENTITY'),
 ('peak effect tail', 'ENTITY'),
 ('stronger', 'ENTITY'),
 ('terpene', 'ENTITY'),
 ('terpene behavior', 'ENTITY'),
 ('tetrad behavior', 'ENTITY'),
 ('weaker positive control win55', 'ENTITY')}


(None, None)

terpene induce cannabimimetic tetrad behavior mouse behavioral analysis data represented radar chart format percent change baseline terpene behavior determined peak effect tail flick selected behavior single time point percent change normalized change 56mg win55 2122 chart chart demonstrates terpene induce tetrad behavior varying efficacy also vary whether effect stronger weaker positive control win55 2122

In [ ]:
nlp = en_core_sci_sm.load()
doc = nlp(text)
displacy_image = displacy.render(doc, jupyter=True,style='ent')
entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
displacy_image, entity_and_label

{('baseline', 'ENTITY'),
 ('behavioral', 'ENTITY'),
 ('cannabimimetic', 'ENTITY'),
 ('change', 'ENTITY'),
 ('chart chart', 'ENTITY'),
 ('chart format', 'ENTITY'),
 ('data', 'ENTITY'),
 ('effect', 'ENTITY'),
 ('efficacy', 'ENTITY'),
 ('flick', 'ENTITY'),
 ('induce', 'ENTITY'),
 ('mouse', 'ENTITY'),
 ('peak effect tail', 'ENTITY'),
 ('stronger', 'ENTITY'),
 ('terpene', 'ENTITY'),
 ('terpene behavior', 'ENTITY'),
 ('tetrad behavior', 'ENTITY'),
 ('weaker positive control win55', 'ENTITY')}


(None, None)

#gene and organism

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bionlp13cg_md-0.5.0.tar.gz
import en_ner_bionlp13cg_md
nlp = en_ner_bionlp13cg_md.load()
doc = nlp(text)
displacy_image = displacy.render(doc, jupyter=True,style='ent')
entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
displacy_image, entity_and_label

{('mouse', 'ORGANISM'), ('win55', 'GENE_OR_GENE_PRODUCT')}


(None, None)

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz
import en_ner_bc5cdr_md
# nlp = en_ner_bc5cdr_md.load()
# doc = nlp(text)
# displacy_image = displacy.render(doc, jupyter=True,style='ent')
# entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
# displacy_image, entity_and_label

     |████████████████████████████████| 120.2 MB 59 kB/s 
     |████████████████████████████████| 6.0 MB 4.3 MB/s 
     |████████████████████████████████| 181 kB 69.5 MB/s 
     |████████████████████████████████| 10.1 MB 44.8 MB/s 
     |████████████████████████████████| 653 kB 29.3 MB/s 
     |████████████████████████████████| 451 kB 57.7 MB/s 
     |████████████████████████████████| 42 kB 999 kB/s 
  Created wheel for en-ner-bc5cdr-md: filename=en_ner_bc5cdr_md-0.5.0-py3-none-any.whl size=120215853 sha256=096d3b5d0651bbe8ee244fba311b05ff65391ac1ebc96ea726786d43ad567604
  Stored in directory: /root/.cache/pip/wheels/34/f2/2e/aa822e510c493b90ab8f7555c48084f946c57121f12a484677
Successfully built en-ner-bc5cdr-md
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Succ

In [ ]:
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_jnlpba_md-0.5.0.tar.gz
import en_ner_jnlpba_md
nlp = en_ner_bc5cdr_md.load()
doc = nlp(text)
displacy_image = displacy.render(doc, jupyter=True,style='ent')
entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
displacy_image, entity_and_label

     |████████████████████████████████| 120.2 MB 30 kB/s 
  Created wheel for en-ner-jnlpba-md: filename=en_ner_jnlpba_md-0.5.0-py3-none-any.whl size=120238325 sha256=b4bcc709fd39cfc776de9985ce51fc4913335795f8e6ae54016cef0f58605cf8
  Stored in directory: /root/.cache/pip/wheels/b2/29/71/fa627302b5a4453dc265ba1897cc8903ef1558f8498171232a
Successfully built en-ner-jnlpba-md


{('win55 2122', 'CHEMICAL'), ('terpene', 'CHEMICAL')}


(None, None)

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_craft_md-0.5.0.tar.gz
import en_ner_craft_md
nlp = en_ner_bc5cdr_md.load()
doc = nlp(text)
displacy_image = displacy.render(doc, jupyter=True,style='ent')
entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
displacy_image, entity_and_label

     |████████████████████████████████| 120.2 MB 1.3 MB/s 
  Created wheel for en-ner-craft-md: filename=en_ner_craft_md-0.5.0-py3-none-any.whl size=120237065 sha256=20f71acfd4cc47dc59b3082e11bdc52e91dcb5c82f18b92d17a35d1a9476bea5
  Stored in directory: /root/.cache/pip/wheels/7b/51/37/16ea575f3400da8f0340109e0b89e43935c8d9e47f927fa411
Successfully built en-ner-craft-md


{('win55 2122', 'CHEMICAL'), ('terpene', 'CHEMICAL')}


(None, None)

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_lg-0.5.0.tar.gz
import en_core_sci_lg
nlp = en_ner_bc5cdr_md.load()
doc = nlp(text)
displacy_image = displacy.render(doc, jupyter=True,style='ent')
entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
displacy_image, entity_and_label

     |████████████████████████████████| 532.3 MB 19 kB/s 
  Created wheel for en-core-sci-lg: filename=en_core_sci_lg-0.5.0-py3-none-any.whl size=532569823 sha256=0694aa25d61b3d1e61fca282e5cf5695cacaa54288ddb839c915b6277108dcda
  Stored in directory: /root/.cache/pip/wheels/c1/69/da/683231a48569a53313ed6530869e4743fe57c344412a56b25f
Successfully built en-core-sci-lg


In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_core_sci_scibert-0.5.0.tar.gz
import en_core_sci_scibert
# nlp = en_ner_bc5cdr_md.load()
# doc = nlp(text)
# displacy_image = displacy.render(doc, jupyter=True,style='ent')
# entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
# displacy_image, entity_and_label

     |████████████████████████████████| 417.6 MB 32 kB/s 
     |████████████████████████████████| 6.0 MB 5.1 MB/s 
     |████████████████████████████████| 51 kB 144 kB/s 
     |████████████████████████████████| 10.1 MB 29.6 MB/s 
     |████████████████████████████████| 181 kB 63.9 MB/s 
     |████████████████████████████████| 451 kB 62.1 MB/s 
     |████████████████████████████████| 653 kB 56.0 MB/s 
     |████████████████████████████████| 42 kB 1.2 MB/s 
     |████████████████████████████████| 3.8 MB 45.1 MB/s 
     |████████████████████████████████| 1.1 MB 37.7 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 596 kB 57.5 MB/s 
     |████████████████████████████████| 895 kB 53.8 MB/s 
     |████████████████████████████████| 6.5 MB 44.8 MB/s 
  Created wheel for en-core-sci-scibert: filename=en_core_sci_scibert-0.5.0-py3-none-any.whl size=417639745 sha256=d877ba98245b3180bd7877b4198532858f282b71741ad3ff709d1f6c91f302d4
  Stored in di

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz
import en_ner_bc5cdr_md

  Using cached https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz (120.2 MB)


In [ ]:
#!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz
import en_ner_bc5cdr_md
tree1 = np.sort(os.listdir('/content/terpene/'))
for l in tree1:
  print('\n',l)
  tree2 = np.sort(os.listdir('/content/terpene/'+ l +'/'+'figure/'))
  for f in tree2:
    print('\n',f)
    text = f_train_data = open('/content/terpene/'+ l +'/'+'figure/' + f +'/caption/raw_text.txt','r')
    reader_train_data = list(csv.reader(f_train_data,delimiter='\t'))
    reader_train_data[0][0]
    text =  clean_text(reader_train_data[0][0])
    nlp = en_ner_bc5cdr_md.load()
    doc = nlp(text)
    displacy_image = displacy.render(doc, jupyter=True,style='ent')
    print('\n')
    print(doc.ents)
    entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
    print(displacy_image)
    print('\n')
    print('-'*50)

NameError: ignored

In [ ]:
text = f_train_data = open('/content/terpene/PMC8050080/figure/Fig5/caption/raw_text.txt','r')
reader_train_data = list(csv.reader(f_train_data,delimiter='\t'))
reader_train_data[0][0]
text =  clean_text(reader_train_data[0][0])
nlp = en_ner_bc5cdr_md.load()
doc = nlp(text)
displacy_image = displacy.render(doc, jupyter=True,style='ent')

entity_and_label = pprint(set([(X.text, X.label_) for X in doc.ents]))
d = set([(X.text, X.label_) for X in doc.ents])
set_labels = set([ X.label_ for X in doc.ents])
for label in set_labels:
  set_content = set([ X.label_  if X.label_ == label, for X in doc.ents])
  if not os.path.exists('\content\'):
    os.makedirs(tree1)
for dd in d:
  print(dd)
print(displacy_image)
d

{('humulene pinene linalool geraniol', 'CHEMICAL'),
 ('hypolocomotion', 'DISEASE'),
 ('terpene', 'CHEMICAL'),
 ('win55 2122', 'CHEMICAL')}
('hypolocomotion', 'DISEASE')
('humulene pinene linalool geraniol', 'CHEMICAL')
('win55 2122', 'CHEMICAL')
('terpene', 'CHEMICAL')
None


{('humulene pinene linalool geraniol', 'CHEMICAL'),
 ('hypolocomotion', 'DISEASE'),
 ('terpene', 'CHEMICAL'),
 ('win55 2122', 'CHEMICAL')}

In [ ]:
set_labels

{'CHEMICAL', 'DISEASE'}

In [ ]:
for dd in d:
  print(dd)

('hypolocomotion', 'DISEASE')
('humulene pinene linalool geraniol', 'CHEMICAL')
('win55 2122', 'CHEMICAL')
('terpene', 'CHEMICAL')
